<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/deep-learning-from-scratch-2/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vecの高速化

## word2vecの改良1
+ 入力層のone-hot表現と重み行列\\( W_{in}\\)の積による計算
+ 中間層と重み行列\\( W_{out} \\)の積およびSoftmaxレイヤの計算

### Embeddingレイヤ
単語IDに該当する行を抜き出すためのレイヤのこと

In [0]:
GPU = True

if GPU:
    import cupy as np
    np.cuda.set_allocator(np.cuda.MemoryPool().malloc)

    print('\033[92m' + '-' * 60 + '\033[0m')
    print(' ' * 23 + '\033[92mGPU Mode (cupy)\033[0m')
    print('\033[92m' + '-' * 60 + '\033[0m\n')
else:
    import numpy as np

------------------------------------------------------------
                       GPU Mode (cupy)
------------------------------------------------------------



### Embeddingレイヤの実装

In [0]:
# import numpy as np
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [0]:
W[2]

array([6, 7, 8])

In [0]:
W[5]

array([15, 16, 17])

In [0]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [0]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        if GPU:
            np.scatter_add(dW, self.idx, dout)
        else:
            np.add.at(dW, self.idx, dout)
        return None

## word2vecの改良2
Negative Sampling

### 中間層以降の計算の問題点
+ 中間層のニューロンと重み行列\\( W_{out} \\)の積
 + 巨大な行列の積は計算に多くの時間を要する。
  + 逆伝播の際も行列の積の計算をするので軽くしたほうがいい。
+ Softmaxレイヤの計算
 + 語彙数が増えるに従い、Softmaxも計算量が増加する。

$$ y_k = \frac{e^{s_k}}{\sum^{1000000}_{i=1}e^{s_i}} $$

### 多値分類から二値分類へ
+ 多値分類を二値分類で近似するというアイデア。
+ これまでの出力層では、すべての単語を対象に計算を行った。Negative Samplingでは一つの単語だけに着目し、そのスコアだけを計算する。そのスコアを確率に変換するために、シグモイド関数を適用する。

### シグモイド関数と交差エントロピー誤差
+ 二値分類の問題をニューラルネットワークで解くには
 + スコアにシグモイド関数を適用して確率を得る
 + 損失関数として交差エントロピー誤差を使用

+ 多値分類の問題をニューラルネットワークで解くには
 + スコアにソフトマックス関数を適用して確率を得る
 + 損失関数として交差エントロピー誤差を使用


シグモイド関数
$$y = \frac{1}{1+e^{-x}}$$


交差エントロピー誤差
$$ L = - (t \log y + (1-t) \log (1-y) ) $$


### 多値分類から二値分類へ（実装編）
+ 中間層のニューロンをhとして、出力側の重み\\( W_{out} \\)の単語に対応する単語ベクトルの内積を計算
+ その出力をSigmoid with Lossレイヤに入れることで、最終的な損失を得る

In [0]:
class EmbeddingDot:
  def __init__(self, W):
    # 4つのメンバ変数
    self.embed = Embedding(W)
    self.params = self.embed.params
    self.grads = self.embed.grads
    self.cache = None # 順伝播の計算結果を一時的に保持

  def forward(self, h, idx):
    target_W = self.embed.forward(idx) # ミニバッチ処理を想定
    out = np.sum(target_W * h, axis=1)

    self.cache = (h, target_W)
    return out

  def backward(self, dout):
    h, target_W = self.cache
    dout = dout.reshape(dout.shape[0], 1)

    dtarget_W = dout * h
    self.embed.backward(dtarget_W)
    dh = dout * target_W
    return dh

### Negative Sampling
+ 正例だけで学習しても意味がないので、負例も必要。ただし、負例を全部含めると数が膨大になる。そこで近似解として負例をいくつかピックアップする。ネガティヴな例（負例）を少数サンプリングして用いる。
+ 正例をターゲットにした場合、損失をもとめ、負例をいくつかサンプリングし、負例に対しても損失をもとめる。それぞれの損失を足し合わせ、その結果を最終的な損失とする。

### Negative Samplingのサンプリング手法
+ コーパスの統計データにもとづいてサンプリングを行う。
 + コーパス中での単語の使用頻度に基づいてサンプリングするには、コーパスから各単語の出現した回数をもとめ、これを確率分布で表す。その確率分布から単語をサンプリングする。
  + レアな単語は出現しにくくなる。その方が精度上望ましい。

In [0]:
# import numpy as np

In [0]:
# 0から9の数字の中から一つの数字をランダムにサンプリング
# np.random.choice(10)

In [0]:
# np.random.choice(10)

In [0]:
# wordsから一つだけランダムにサンプリング
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
# np.random.choice(words)

In [0]:
# 5つだけランダムサンプリング（重複あり）
# np.random.choice(words, size=5)

In [0]:
# 5つだけランダムサンプリング（重複なし）
# np.random.choice(words, size=5, replace=False)

In [0]:
# 確率分布に従ってサンプリング
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
# np.random.choice(words, p=p)

word2vecのNegative Sampling
$$ P'(W_i) = \frac{P(w_i)^{0.75}}{\sum_j^n P(w_j)^{0.75}} $$

出現確率の低い単語を見捨てないようにするため。0.75に理論的な意味付けはない。

In [0]:
# p = [0.7, 0.29, 0.01]
# new_p = np.power(p, 0.75)
# new_p /= np.sum(new_p)
# print(new_p)

In [0]:
# UnigramSampler

import collections

class UnigramSampler:
  def __init__(self, corpus, power, sample_size):
    self.sample_size = sample_size
    self.vocab_size = None
    self.word_p = None

    counts = collections.Counter()
    for word_id in corpus:
      counts[word_id] += 1

    vocab_size = len(counts)
    self.vocab_size = vocab_size

    self.word_p = np.zeros(vocab_size)
    for i in range(vocab_size):
      self.word_p[i] = counts[i]

    self.word_p = np.power(self.word_p, power)
    self.word_p /= np.sum(self.word_p)

  def get_negative_sample(self, target):
    batch_size = target.shape[0]

    if not GPU:
      negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

      for i in range(batch_size):
        p = self.word_p.copy()
        target_idx = target[i]
        p[target_idx] = 0
        p /= p.sum()
        negative_sample[i, :] = np.random.choice((self.vocab_size, self.sample_size),replace=False, p=p)
    else:
      # GPU(cupy)で計算するときは、速度を優先
      # 負例にターゲットが含まれるケースがある
      negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),replace=True, p=self.word_p)

    return negative_sample

In [0]:
# # GPU = False

# corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
# power = 0.75
# sample_size = 2

# sampler = UnigramSampler(corpus, power, sample_size)
# target = np.array([1, 3, 0])
# negative_sample = sampler.get_negative_sample(target)
# print(negative_sample)

### Negative Samplingの実装

In [0]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx


def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
  if t.size == y.size:
    t = t.argmax(axis=1)

  batch_size = y.shape[0]

  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.loss = None
        self.y = None  # sigmoidの出力
        self.t = None  # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = 1 / (1 + np.exp(-x))

        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = (self.y - self.t) * dout / batch_size
        return dx

In [0]:
class NegativeSamplingLoss:
  def __init__(self, W, corpus, power=0.75, sample_size=5):
    import numpy as nump
    self.sample_size = sample_size
    self.sampler = UnigramSampler(corpus, power, sample_size)
    # 正例用のレイヤをひとつ、負例用のレイヤをsample_size個だけ生成する
    self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1) ]
    self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
    self.params, self.grads = [], []
    for layer in self.embed_dot_layers:
      self.params += layer.params
      self.grads += layer.grads

  # 中間層のニューロンhと正例のターゲットを受け取る
  def forward(self, h, target):
    import numpy as nump
    batch_size = target.shape[0]
    negative_sample = self.sampler.get_negative_sample(target)

    # 正例のフォワード
    score = self.embed_dot_layers[0].forward(h, target)
    correct_label = nump.ones(batch_size, dtype=nump.int32)

    # 損失をもとめる
    loss = self.loss_layers[0].forward(score, correct_label)

    # 負例のフォワード
    negative_label = nump.zeros(batch_size, dtype=nump.int32)
    for i in range(self.sample_size):
      negative_target = negative_sample[:, i]
      score = self.embed_dot_layers[1 + i].forward(h, negative_target)
      # 損失をもとめる
      loss += self.loss_layers[1 + i].forward(score, negative_label)

    return loss

  # 逆伝播
  def backward(self, dout=1):
    dh = 0
    for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
      dscore = l0.backward(dout)
      dh += l1.backward(dscore)
    return dh

## 改良版word2vecの学習

### CBOWモデルの実装

In [0]:
class CBOW:
  def __init__(self, vocab_size, hidden_size, window_size, corpus):
    V, H = vocab_size, hidden_size

    # 重みの初期化
    W_in = 0.01 * np.random.randn(V, H).astype('f')
    W_out = 0.01 * np.random.randn(V, H).astype('f')

    # レイヤの生成
    self.in_layers = []
    for i in range(2 * window_size):
      layer = Embedding(W_in) # Embeddingレイヤを使用
      self.in_layers.append(layer)
    self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

    # すべての重みと勾配を配列にまとめる
    layers = self.in_layers + [self.ns_loss]
    self.params, self.grads = [], []
    for layer in layers:
      self.params += layer.params
      self.grads += layer.grads

    # メンバ変数に単語の分散表現を設定
    self.word_vecs = W_in

  def forward(self, contexts, target):
    h = 0
    for i, layer in enumerate(self.in_layers):
      h += layer.forward(contexts[:, i])
    h *= 1 / len(self.in_layers)
    loss = self.ns_loss.forward(h, target)
    return loss

  def backward(self, dout=1):
    dout = self.ns_loss.backward(dout)
    dout *= 1 / len(self.in_layers)
    for layer in self.in_layers:
      layer.backward(dout)
    return None

### CBOWモデルの学習コード

In [0]:
import pickle
import time

def create_contexts_target(corpus, window_size=1):
  target = corpus[window_size:-window_size]
  contexts = []

  for idx in range(window_size, len(corpus) - window_size):
    cs = []
    for t in range(-window_size, window_size + 1):
      if t == 0:
        continue
      cs.append(corpus[idx + t])
    contexts.append(cs)

  return np.array(contexts), np.array(target)

def remove_duplicate(params, grads):
    '''
    パラメータ配列中の重複する重みをひとつに集約し、
    その重みに対応する勾配を加算する
    '''
    params, grads = params[:], grads[:]

    while True:
      find_flg = False
      L = len(params)

      for i in range(0, L - 1):
        for j in range(i + 1, L):
          # 重みを共有する場合
          if params[i] is params[j]:
            grads[i] += grads[j] # 勾配の加算
            find_flg = True
            params.pop(j)
            grads.pop(j)
          # 転置行列として重みを共有する場合
          elif params[i].ndim == 2 and params[j].ndim == 2 \
              and params[i].T.shape == params[j].shape \
              and np.all(params[i].T == params[j]):
            grads[i] += grads[j].T
            find_flg = True
            params.pop(j)
            grads.pop(j)

          if find_flg:
            break

        if find_flg:
          break

      if not find_flg:
        break

    return params, grads


def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate


def to_cpu(x):
    import numpy
    if type(x) == numpy.ndarray:
        return x
    return np.asnumpy(x)


def to_gpu(x):
    import cupy
    if type(x) == cupy.ndarray:
        return x
    return cupy.asarray(x)


class Trainer:
  def __init__(self, model, optimizer):
    self.model = model
    self.optimizer = optimizer
    self.loss_list = []
    self.eval_interval = None
    self.current_epoch = 0

  def fit(self, x, t, max_epoch=10, batch_size=32, max_grad=None,eval_interval=20):
    data_size = len(x)
    max_iters = data_size // batch_size
    self.eval_interval = eval_interval
    model, optimizer = self.model, self.optimizer
    total_loss = 0
    loss_count = 0

    start_time = time.time()
    for epoch in range(max_epoch):
      # シャッフル
      idx = np.random.permutation(np.arange(data_size))
      x = x[idx]
      t = t[idx]

      for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1)*batch_size]
        batch_t = t[iters*batch_size:(iters+1)*batch_size]

        # 勾配を求め、パラメータを更新
        loss = model.forward(batch_x, batch_t)
        model.backward()
        params, grads = remove_duplicate(model.params, model.grads) # 共有された重みを1つに集約

        if max_grad is not None:
          clip_grads(grads, max_grad)
        optimizer.update(params, grads)
        total_loss += loss
        loss_count += 1

        # 評価
        if (eval_interval is not None) and (iters % eval_interval) == 0:
          avg_loss = total_loss / loss_count
          elapsed_time = time.time() - start_time
          print('| epoch %d | iter %d / %d | time %d[s] | loss %.2f' % (self.current_epoch + 1, iters + 1, max_iters, elapsed_time, avg_loss))
          self.loss_list.append(float(avg_loss))
          total_loss, loss_count = 0, 0

      self.current_epoch += 1

  def plot(self, ylim=None):
      x = np.arange(len(self.loss_list))
      if ylim is not None:
        plt.ylim(*ylim)
      plt.plot(x, self.loss_list, label='train')
      plt.xlabel('iterations (x' + str(self.eval_interval) + ')')
      plt.ylabel('loss')
      plt.show()


class Adam:
    '''
    Adam (http://arxiv.org/abs/1412.6980v8)
    '''
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
      self.lr = lr
      self.beta1 = beta1
      self.beta2 = beta2
      self.iter = 0
      self.m = None
      self.v = None

    def update(self, params, grads):
      if self.m is None:
        self.m, self.v = [], []
        for param in params:
          self.m.append(np.zeros_like(param))
          self.v.append(np.zeros_like(param))

      self.iter += 1
      lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

      for i in range(len(params)):
        self.m[i] += (1 - self.beta1)*(grads[i] - self.m[i])
        self.v[i] += (1 - self.beta2)*(grads[i]**2 - self.v[i])

        params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7 )

In [0]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [0]:
import sys
import os
sys.path.append('..')
try:
    import urllib.request
except ImportError:
    raise ImportError('Use Python3!')
import pickle
# import numpy as np


url_base = 'https://raw.githubusercontent.com/tomsercu/lstm/master/data/'
key_file = {
    'train':'ptb.train.txt',
    'test':'ptb.test.txt',
    'valid':'ptb.valid.txt'
}
save_file = {
    'train':'ptb.train.npy',
    'test':'ptb.test.npy',
    'valid':'ptb.valid.npy'
}
vocab_file = 'ptb.vocab.pkl'

dataset_dir = os.path.dirname(os.path.abspath('/content'))

def _download(file_name):
    file_path = dataset_dir + '/' + file_name
    if os.path.exists(file_path):
        return

    print('Downloading ' + file_name + ' ... ')

    try:
        urllib.request.urlretrieve(url_base + file_name, file_path)
    except urllib.error.URLError:
        import ssl
        ssl._create_default_https_context = ssl._create_unverified_context
        urllib.request.urlretrieve(url_base + file_name, file_path)

    print('Done')

def load_vocab():
    vocab_path = dataset_dir + '/' + vocab_file

    if os.path.exists(vocab_path):
        with open(vocab_path, 'rb') as f:
            word_to_id, id_to_word = pickle.load(f)
        return word_to_id, id_to_word

    word_to_id = {}
    id_to_word = {}
    data_type = 'train'
    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name

    _download(file_name)

    words = open(file_path).read().replace('\n', '<eos>').strip().split()

    for i, word in enumerate(words):
        if word not in word_to_id:
            tmp_id = len(word_to_id)
            word_to_id[word] = tmp_id
            id_to_word[tmp_id] = word

    with open(vocab_path, 'wb') as f:
        pickle.dump((word_to_id, id_to_word), f)

    return word_to_id, id_to_word



def load_data(data_type='train'):
    '''
        :param data_type: データの種類：'train' or 'test' or 'valid (val)'
        :return:
    '''
    if data_type == 'val': data_type = 'valid'
    save_path = dataset_dir + '/' + save_file[data_type]

    word_to_id, id_to_word = load_vocab()

    if os.path.exists(save_path):
        corpus = np.load(save_path)
        return corpus, word_to_id, id_to_word

    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name
    _download(file_name)

    words = open(file_path).read().replace('\n', '<eos>').strip().split()
    corpus = np.array([word_to_id[w] for w in words])

    np.save(save_path, corpus)
    return corpus, word_to_id, id_to_word

In [0]:
# ハイパーパラメータの設定
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# データの読み込み
corpus, word_to_id, id_to_word = load_data('train')
vocab_size = len(word_to_id)

In [0]:
import numpy as np

In [0]:
contexts, target = create_contexts_target(corpus, window_size)

In [0]:
import cupy as np

In [0]:
if GPU:
  contexts, target = to_gpu(contexts), to_gpu(target)

# モデルなどの生成
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [0]:
# 学習開始
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 後ほど利用できるように、必要なデータを保存
word_vecs = model.word_vecs

if GPU:
  word_vecs = to_cpu(word_vecs)

params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word

pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
  pickle.dump(params, f, -1)

TypeError: ignored

GPU使わないと半日くらいかかるらしい。

In [0]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
  # STEP1 クエリを取り出す
  if query not in word_to_id:
    print('%s is not found' % query)
    return

  print('\n[query] ' + query)
  query_id = word_to_id[query]
  query_vec = word_matrix[query_id]

  # STEP2 コサイン類似度の算出
  vocab_size = len(id_to_word)
  similarity = np.zeros(vocab_size)
  for i in range(vocab_size):
    similarity[i] = cos_similarity(word_matrix[i], query_vec)

  # STEP3 コサイン類似度の結果から、その値を高い順に出力
  count = 0
  for i in (-1 * similarity).argsort():
    if id_to_word[i] == query:
      continue
    print(' %s: %s' % (id_to_word[i], similarity[i]))

    count += 1
    if count >= top:
      return

In [0]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
  params = pickle.load(f)
  word_vecs = params['word_vecs']
  word_to_id = params['word_to_id']
  id_to_word = params['id_to_word']

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
  most_similar(query, word_to_id, id_to_word, word_vecs, top=5)